選擇最少步的
* 找出還沒填square
* check candidate
> no cadidate -> violation，此路不通
> 找出最少 cadidate 的 square，依序填入
> 繼續找下一個最少的 square，直到路不通



In [6]:
import timeit

def reset():
    global p1,p2,p3,p4,p5,p6,p7
    # Guardian hard
    p1 = ['005000900',
                    '004690100',
                    '790000000',
                    '010200003',
                    '070006080',
                    '000014602',
                    '230008000',
                    '000050007',
                    '000403010']
    p1 = [[int(v) for v in x] for x in p1]


    p2 = [[5,3,0,0,7,0,0,0,0],
            [6,0,0,1,9,5,0,0,0],
            [0,9,8,0,0,0,0,6,0],
            [8,0,0,0,6,0,0,0,3],
            [4,0,0,8,0,3,0,0,1],
            [7,0,0,0,2,0,0,0,6],
            [0,6,0,0,0,0,2,8,0],
            [0,0,0,4,1,9,0,0,5],
            [0,0,0,0,8,0,0,7,9],]

    p3 = \
            [[5,4,0,0,2,0,8,0,6],
            [0,1,9,0,0,7,0,0,3],
            [0,0,0,3,0,0,2,1,0],
            [9,0,0,4,0,5,0,2,0],
            [0,0,1,0,0,0,6,0,4],
            [6,0,4,0,3,2,0,8,0],
            [0,6,0,0,0,0,1,9,0],
            [4,0,2,0,0,9,0,0,5],
            [0,9,0,0,7,0,4,0,2],]

    p4 = \
            [[0,0,0,8,0,0,4,2,0],
            [5,0,0,6,7,0,0,0,0],
            [0,0,0,0,0,9,0,0,5],
            [7,4,0,1,0,0,0,0,0],
            [0,0,9,0,3,0,7,0,0],
            [0,0,0,0,0,7,0,4,8],
            [8,0,0,4,0,0,0,0,0],
            [0,0,0,0,9,8,0,0,3],
            [0,9,5,0,0,3,0,0,0],]

    # NYTimes Sudoku May 14, 2021
    p5 = ['000040000',
             '000000000',
             '160820900',
             '509000008',
             '000000100',
             '000093604',
             '010030070',
             '020500040',
             '750000001']
    p5 = [[int(v) for v in x] for x in p5]

    # NYTimes Sudoku May 14, 2021, easy
    p6 = ['030417056','570630040','940000013',
          '293500600','000261300','008700405',
          '005376000','009000807','100002004']
    p6 = [[int(v) for v in x] for x in p6]


    # NYTimes Sudoku May 14, 2021, medium
    p7 = ['059000702','001004059','000600300',
          '000010008','500800000','060000130',
          '400030000','007002000','000007260']
    p7 = [[int(v) for v in x] for x in p7]

In [728]:
%%timeit
test_speed(p2, display_on = False)

23.4 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def test_speed(board, display_on=True):
    reset()
    solve(board, display_on)

In [11]:
reset()
solution = []
for itm in [p1,p2,p3,p4,p5,p6,p7]:
    solve(itm,display_on=False)
    print(is_complete(solution), is_legit(solution))

True True
True True
True True
True True
True True
True True
True True


In [9]:
reset()
solve(p4, display_on=True)


    -------------------------------------
    | 9 | 6 | 3 | 8 | 5 | 1 | 4 | 2 | 7 |
    -------------------------------------
    | 5 | 1 | 4 | 6 | 7 | 2 | 3 | 8 | 9 |
    -------------------------------------
    | 2 | 7 | 8 | 3 | 4 | 9 | 6 | 1 | 5 |
    -------------------------------------
    | 7 | 4 | 2 | 1 | 8 | 5 | 9 | 3 | 6 |
    -------------------------------------
    | 6 | 8 | 9 | 2 | 3 | 4 | 7 | 5 | 1 |
    -------------------------------------
    | 3 | 5 | 1 | 9 | 6 | 7 | 2 | 4 | 8 |
    -------------------------------------
    | 8 | 3 | 7 | 4 | 1 | 6 | 5 | 9 | 2 |
    -------------------------------------
    | 4 | 2 | 6 | 5 | 9 | 8 | 1 | 7 | 3 |
    -------------------------------------
    | 1 | 9 | 5 | 7 | 2 | 3 | 8 | 6 | 4 |
    -------------------------------------


True

In [5]:


# Try to solve board repeated issue
# 比較符合 back tracking 的精神

solution = [] # store solution

def solve(board, display_on=True):
    global solution
    # 重新建立新的 board，才不會修改到原 board
    board = [[v for v in x] for x in board]
    # board = copy.deepcopy(board) -> slower
    
    # display option
    if display_on:
        display(board)
    
    # 不需要檢查是否有重複解，因為填入的數字是從 candidate 抓出來的，本來就不會重複
    # backtrack 的精神就是 從最後一步如果填滿了開始推回去想
    # 檢查是否有重複解，有的話為 false
    #if is_legit(board)==False:
    #    return False
    
    # 是否完全填滿了？
    if is_complete(board):
        solution = board
        return True
    
    # 獲得 candidate
    ava_candidate_original = check_ava_candidate(board)
    
    # 還沒填入數值的位置已經沒有步了的話，那此路不通
    if is_alive(board,ava_candidate_original)==False:
        return False

    # 依照 candidate 多寡排序
    # slow process, 先刪除空白者
    ava_candidate = {key:value for key,value in ava_candidate_original.items() if (value)!=[]}
    
    # 求最小 candidate_list
    #location, candidate_list = get_min_list(ava_candidate)
    ava_candidate = sorted(ava_candidate.items(),key=lambda x:len(x[1]))
    location, candidate_list = ava_candidate[0]
    
    # 一定從最小的開始
    # 而且每次都會重新 call solve，所以不需要 iteration 每一個解（但這樣效率其實比較慢...?）
    r,c = location

    # 已經刪掉沒有 candidate 的 item，包含已經完成的地方，所以不需要再判斷是否 board[r][c]!=0:
    # 只有一組解，則直接填入，並往下一步
    if len(candidate_list)==1:
        
        # 賦值 board，下一步
        temp_board = [[v for v in b] for b in board]
        board[r][c]=candidate_list[0]
        
        if solve(board, display_on)==True:
            return True
        else:
            board = temp_board
            return False
        
    # 多組解。會有需要 backtracking 退回另一個解的，都是在這邊
    else:
        for candidate in candidate_list:
            # 暫存解，提供 backtracking
            temp_board = [[v for v in b] for b in board]
            board[r][c]=candidate
            
            # 如果有解，return True
            if solve(board, display_on)==True:
                return True
            else:
                # 如果往下無解，則回覆數值
                board = temp_board

    # code 不會跑到這裡來
    #return board
    
    
    

## Functions

In [1]:
def check_ava_candidate(board):
    ava_dict = {}
    
    for r in range(9):
        for c in range(9):
            # 如果已經有數值了，那無 available candidate
            if board[r][c]!=0:
                ava_dict[(r,c)]=[]
            else:
                candidates = [1,2,3,4,5,6,7,8,9]
                
                # Remove same row numbers
                for i in range(9):
                    try:
                        candidates.remove(board[r][i])
                    except:
                        pass
                # Remove same col numbers
                for i in range(9):
                    try:
                        candidates.remove(board[i][c])
                    except:
                        pass
                # Remove same 3x3 sector numbers
                r_base = r//3*3
                c_base = c//3*3
                for r_sector in range(r_base,r_base+3):
                    for c_sector in range(c_base, c_base+3):
                        try:
                            candidates.remove(board[r_sector][c_sector])
                        except:
                            pass
                ava_dict[(r,c)] = candidates
    return ava_dict

    
def is_complete(board):
    # 判斷是否得解
    for i in range(9):
        for j in range(9):
            if board[i][j]==0:
                return False
    
    return True


def is_alive(board, ava_candidate):
    
    for i in range(9):
        for j in range(9):
            # 如果有還沒被填入數值的地方 (value==0)，卻沒有 candidate 了，表示此解不通
            if board[i][j]==0 and len(ava_candidate[(i,j)])==0:
                return False
    return True
                            


def is_legit(board):
    #for i in range(9):
    #    for j in range(9):
    #        # 如果有還沒被填入數值的地方 (value==0)，卻沒有 candidate 了，表示此解不通
    #        if board[i][j]==0 and len(ava_candidate[(i,j)])==0:
    #            return False
                            
    # 如果有重複數值，表示錯誤
    for itm in board:
        for i in range(1,10):
            if itm.count(i)>1:
                return False
    for col in range(9):
        temp_col = [board[row][col] for row in range(9)]
        for i in range(1,10):
            if temp_col.count(i)>1:
                return False
    # 3x3 box
    for r in range(3):
        for c in range(3):
            temp_box = [board[3*r+i][3*c+j] for i in range(3) for j in range(3)]
            for i in range(1,10):
                if temp_box.count(i)>1:
                    return False
                    
    return True



from IPython.display import clear_output
import time
def display(board):
    clear_output(wait='False')
    time.sleep(0.01)
    
    # construct each row numbers
    p = []
    for i in range(len(board)):
        add_row = [' '+str(n)+' ' if n!=0 else '   ' for n in board[i]]
        p.append(add_row)
        
    # add border
    row_str = []
    for i in range(len(board)):
        add_str = '|' + '|'.join(p[i]) + '|'
        row_str.append(add_str)
        
    ret =\
    f"""
    -------------------------------------
    {row_str[0]}
    -------------------------------------
    {row_str[1]}
    -------------------------------------
    {row_str[2]}
    -------------------------------------
    {row_str[3]}
    -------------------------------------
    {row_str[4]}
    -------------------------------------
    {row_str[5]}
    -------------------------------------
    {row_str[6]}
    -------------------------------------
    {row_str[7]}
    -------------------------------------
    {row_str[8]}
    -------------------------------------"""
    print(ret)

    



    

## Other function

In [ ]:
# not used, because it's slower
def get_min_list(ava_candidate):
    for minimal in range(1,10):
        for location, candidate_list in ava_candidate.items():
            #print(location, candidate_list)
            if len(candidate_list)==minimal:
                return location, candidate_list

In [765]:
## ADD more infer -> 不會比較快

# Try to solve board repeated issue
# 比較符合 back tracking 的精神


solution = [] # store solution

def solve(board, display_on=True):
    global solution
    # 重新建立新的 board，才不會修改到原 board
    board = [[v for v in x] for x in board]
    # board = copy.deepcopy(board) -> slower
    
    # display option
    if display_on:
        display(board)
    
    # 不需要檢查是否有重複解，因為填入的數字是從 candidate 抓出來的，本來就不會重複
    # backtrack 的精神就是 從最後一步如果填滿了開始推回去想
    # 檢查是否有重複解，有的話為 false
    #if is_legit(board)==False:
    #    return False
    
    # 是否完全填滿了？
    if is_complete(board):
        solution = board
        return True
    
    # 獲得 candidate
    ava_candidate_original = check_ava_candidate(board)
    
    # 還沒填入數值的位置已經沒有步了的話，那此路不通
    if is_alive(board,ava_candidate_original)==False:
        return False

    # 依照 candidate 多寡排序
    # slow process, 先刪除空白者
    ava_candidate = {key:value for key,value in ava_candidate_original.items() if (value)!=[]}
    
    # 求最小 candidate_list
    #location, candidate_list = get_min_list(ava_candidate)
    ava_candidate = sorted(ava_candidate.items(),key=lambda x:len(x[1]))
    location, candidate_list = ava_candidate[0]
    
    # 一定從最小的開始
    # 而且每次都會重新 call solve，所以不需要 iteration 每一個解（但這樣效率其實比較慢...?）
    r,c = location

    # 已經刪掉沒有 candidate 的 item，包含已經完成的地方，所以不需要再判斷是否 board[r][c]!=0:
    # 只有一組解，則直接填入，並往下一步
    if len(candidate_list)==1:
        
        # 賦值 board，下一步
        temp_board = [[v for v in b] for b in board]
        board[r][c]=candidate_list[0]
        
        if solve(board, display_on)==True:
            return True
        else:
            board = temp_board
            return False
        
    # 多組解。會有需要 backtracking 退回另一個解的，都是在這邊
    else:
        check_infer = infer(board,location,ava_candidate_original)
        if check_infer is not None:
            temp_board = [[v for v in b] for b in board]
            board[r][c]=check_infer

            if solve(board, display_on)==True:
                return True
            else:
                board = temp_board
                return False
        
        for candidate in candidate_list:
            # 暫存解，提供 backtracking
            temp_board = [[v for v in b] for b in board]
            board[r][c]=candidate
            
            # 如果有解，return True
            if solve(board, display_on)==True:
                return True
            else:
                # 如果往下無解，則回覆數值
                board = temp_board

    # code 不會跑到這裡來
    #return board
    
    
### 如果去掉橫排，直排的其他人的candidate，只剩下一個的話，那麼就只有填入這個數字了
### 實際上不會用到

def infer(board, position, ava_candidate_original):
    
    r, c = position
    
    # check row
    row_candidate_list = ava_candidate_original[position]
    for i in range(9):
        for itm in ava_candidate_original[(r,i)]:
            try:
                row_candidate_list.revmoe(itm)
            except:
                pass
    if len(row_candidate_list)==1:
        return row_candidate_list[0]

    ## check column
    #col_candidate_list = set(ava_candidate_original[position])
    for i in range(9):
        for itm in ava_candidate_original[(i,c)]:
            try:
                row_candidate_list.revmoe(itm)
            except:
                pass
    if len(row_candidate_list)==1:
        print('y')
        return row_candidate_list[0]
        
    return None


    

In [771]:
%%timeit
test_speed(p7, display_on = False)

373 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [674]:
# new_ 更進一步優化

def solve(board, display_on=True):
    # 重新建立新的 board，才不會修改到原 board
    board = [[v for v in x] for x in board]
    
    if display_on:
        display(board)
    
    
    # 不需要檢查是否有重複解，因為填入的數字是從 candidate 抓出來的，本來就不會重複
    # backtrack 的精神就是 從最後一步如果填滿了開始推回去想
    # 檢查是否有重複解，有的話為 false
    #if is_legit(board)==False:
    #    return False
    
    # 是完全填滿了？
    if is_complete(board):
        return board
    
    # 獲得 candidate
    ava_candidate = check_ava_candidate(board)
    
    
    # 還沒填入數值的位置已經沒有步了的話，那此路不通
    if is_alive(board,ava_candidate)==False:
        return False

    # 依照 candidate 多寡排序
    # slow process, 先刪除空白者
    ava_candidate = {key:value for key,value in ava_candidate.items() if value!=[]}
    
    # 求最小 candidate_list
    #location, candidate_list = get_min_list(ava_candidate)
    ava_candidate = sorted(ava_candidate.items(),key=lambda x:len(x[1]))
    location, candidate_list = ava_candidate[0]
    
    # 一定從最小的開始
    # 而且每次都會重新 call solve，所以不需要 iteration 每一個解（但這樣效率其實比較慢...?）
    r,c = location

    # 已經刪掉沒有 candidate 的 item，包含已經完成的地方，所以不需要再判斷是否 board[r][c]!=0:
    # 只有一組解，則直接填入，並往下一步
    if len(candidate_list)==1:
        # 賦值 board，下一步
        board[r][c]=candidate_list[0]
        return_board = solve(board, display_on)
        if return_board==False:
            return False
        
    # 多組解。會有需要 backtracking 退回另一個解的，都是在這邊
    else:
        for candidate in candidate_list:
            board[r][c]=candidate
            return_board = solve(board, display_on)

            # 如果此路不通，往下一個解進行
            if return_board==False:
                continue
            else:
                return return_board

    return return_board
    
    
    

In [646]:
# new, with iteration

def solve(board, display_on=True):
    # 重新建立新的 board，才不會修改到原 board
    board = [[v for v in x] for x in board]
    
    if display_on:
        display(board)
    
    # 檢查是否有重複解，有的話為 false
    #if is_legit(board)==False:
    #    return False
    
    # 是完全填滿了？
    #if is_complete(board):
    #    return board
    
    # 獲得 candidate
    ava_candidate = check_ava_candidate(board)
    
    # 還沒填入數值的位置已經沒有步了的話，那此路不通
    if is_alive(board,ava_candidate)==False:
        return False
    
    # 依照 candidate 多寡排序
    # slow process, 先刪除空白者
    ava_candidate = {key:value for key,value in ava_candidate.items() if len(value)!=0}
    ava_candidate = sorted(ava_candidate.items(),key=lambda x:len(x[1]))
    
    for location, candidate_list in ava_candidate:
        r,c = location
        # return_board = [[v for v in c] for c in board]
        # 已經刪掉沒有 candidate 的 item，包含已經完成的地方，所以不需要再判斷是否 board[r][c]!=0:
        # 只有一組解，則直接填入，並往下一步
        if len(candidate_list)==1:
            # 賦值 board，下一步
            board[r][c]=candidate_list[0]
            return_board = solve(board, display_on)
            if return_board==False:
                return False
        # 多組解。會有需要 backtracking 的，都是在這邊
        else:
            for candidate in candidate_list:
                board[r][c]=candidate
                return_board = solve(board, display_on)
                
                # 如果此路不通，往下一個解進行
                if return_board==False:
                    continue
                else:
                    return return_board
        return return_board
        
        

    return board
    
    
    

In [464]:
# new 1

def solve(board, display_on=True):
    # 重新建立新的 board，才不會修改到原 board
    board = [[v for v in x] for x in board]
    
    if display_on:
        display(board)
    
    # 檢查是否有重複解，有的話為 false
    if is_legit(board)==False:
        return False
    
    if is_complete(board):
        return board
    
    # 獲得 candidate
    ava_candidate = check_ava_candidate(board)
    
    
    # 依照 candidate 多寡排序
    # slow
    sorted_candidate = sorted(ava_candidate.items(),key=lambda x:len(x[1]))

    for location, candidate_list in sorted_candidate:
        r,c = location
        
        # 如果已經有填入數值，則 pass
        if board[r][c]!=0:
            continue
        # 如果需要填入數值但是沒有 candidate，表示此路不通
        # 在這邊檢查不合格
        # if return_board[r][c]==0
        elif len(candidate_list)==0:
            return False
        # 只有一組解，則直接填入，並往下一步
        elif len(candidate_list)==1:
            # 賦值 board，下一步
            board[r][c]=candidate_list[0]
            return_board = solve(board, display_on)
            if return_board==False:
                return False
        # 多組解。會有需要 backtracking 的，都是在這邊
        else:
            for candidate in candidate_list:
                board[r][c]=candidate
                return_board = solve(board, display_on)
                
                # 如果此路不通，往下一個解進行
                if return_board==False:
                    continue
                else:
                    return return_board
        return return_board
        
        

    return return_board
    
    
    